In [1]:
# Mounting drive

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
! unzip gdrive/My\ Drive/fer2013.zip

Archive:  gdrive/My Drive/fer2013.zip
  inflating: fer2013.csv             


In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:
num_classes = 7 # angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 256
epochs = 5

In [16]:
with open("fer2013.csv") as f:
  content = f.readlines()
 
lines = np.array(content)
 
num_of_instances = lines.size
print("number of instances: ",num_of_instances)

number of instances:  35888


In [0]:
# Initialising train and test sets

x_train, y_train, x_test, y_test = [], [], [], []

In [0]:
for i in range(1, num_of_instances):
  
  try:
    emotion, img, usage = lines[i].split(",")
    
    val = img.split(" ")
    pixels = np.array(val, 'float32')
    
    if 'Training' in usage:
      y_train.append(emotion)
      x_train.append(pixels)
      
    elif 'PublicTest' in usage:
      y_test.append(emotion)
      x_test.append(pixels)
      
  except:
    print("", end="")

In [0]:
import torch.utils.data as data
import pandas as pd

In [0]:
class FerDataset(data.Dataset):
  
  def __init__(self, split='Training', transform=None, csv_path):
    
    self.transform = transform
    
    self.split = split
    
    self.data = pd.read_csv('fer2013.csv')
    
    if self.split == 'Training':
      
      self.train_data = self.data['pixels']
      self.train_labels = self.data['emotion']
      self.train_data = np.asarray(self.train_data)
      self.train_data = self.train_data.reshape((28709, 48, 48))
      
    
    elif self.split == 'PublicTest':
      
      self.PublicTest_data = self.data['pixels']
      self.Publictest_labels = self.data['emotion']
      self.PublicTest_data = np.asarray(self.PublicTest_data)
      self.PublicTest_data = self.PublicTest_data.reshape((3589, 48, 48))
   
  
    else:
      
      self.PrivateTest_data = self.data['pixels']
      self.Privatetest_labels = self.data['emotion']
      self.PrivateTest_data = np.asarray(self.PrivateTest_data)
      self.PrivateTest_data = self.PrivateTest_data.reshape((3589, 48, 48))